# Project 1: 使用Numpy进行文本的四分类问题

## 步骤1: 实现数据读取

### 导入pandas读取

In [1]:
import pandas as pd 

### 读入数据

In [2]:
train_data = pd.read_csv("./train.tsv",header=0,delimiter="\t")
test_data = pd.read_csv("./test.tsv",header=0,delimiter="\t")

### 数据内容

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


### 数据标题

In [4]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


### 计算数据比例

In [5]:
print(train_data.Sentiment.value_counts()/train_data.Sentiment.count())

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64


In [6]:
X = train_data['Phrase']
Y = train_data['Sentiment']
test_X = test_data['Phrase']
data_train = list(X)
label_train = list(Y)

In [7]:
len(data_train)

156060

### 实现分词

In [8]:
## 实现文本的分词
def get_word(text):
    return [word.lower() for word in text.split(' ')]

def get_whole_word(data):
    return[get_word(text) for text in data]

word_list = get_whole_word(data_train)
print(word_list[0])

['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.']


### 实现词袋(N-gram)
实现二元语法。

In [9]:
def get_2gram(word_list):
    word_bag = []
    for words in word_list:
        if len(words)==1:
            word_bag.append(words)
        else:
            ngram = [(a+' '+b) for a,b in zip(words[:-1],words[1:])]
            word_bag.append(ngram)
    return word_bag
word_bag = get_2gram(word_list)
print(word_bag[0])

['a series', 'series of', 'of escapades', 'escapades demonstrating', 'demonstrating the', 'the adage', 'adage that', 'that what', 'what is', 'is good', 'good for', 'for the', 'the goose', 'goose is', 'is also', 'also good', 'good for', 'for the', 'the gander', 'gander ,', ', some', 'some of', 'of which', 'which occasionally', 'occasionally amuses', 'amuses but', 'but none', 'none of', 'of which', 'which amounts', 'amounts to', 'to much', 'much of', 'of a', 'a story', 'story .']


### 实现可迭代的对象
`idx_to_char`：一个列表，实现的是从迭代对象中抽取词袋作为一个总列表。

`char_to_idx`：一个字典，实现的是词袋和出现的位置的对应（标签号）.

In [10]:
import collections
def get_vocab(word_bag):
    counter = collections.Counter([x for sublist in word_bag for x in sublist])
    to_char = [item[0] for item in counter.items()]
    to_idx = dict([(char,idx)for idx,char in enumerate(to_char)])
    return to_char,to_idx
idx_to_char,char_to_idx = get_vocab(word_bag)
print(len(idx_to_char))
print(char_to_idx['jumbled fantasy'])

100664
83497


### 实现数据集的切割
按照`测试集`和`训练集`为1:6的比例进行拆分。

In [11]:
import random # 实现随机打乱原始的dict
original_dict = list(zip(data_train,label_train))
random.shuffle(original_dict)
data_train[:],label_train[:] = zip(*original_dict)
len_train = int(len(original_dict) * 0.83)
train_phrase = data_train[:len_train]
train_label = label_train[:len_train]
test_phrase = data_train[len_train:]
test_label = label_train[len_train:]
print(len(train_phrase),len(train_label),len(test_phrase),len(test_label))

129529 129529 26531 26531


### 实现测试集和训练集的ngram词袋

In [12]:
train_2gram = get_2gram(get_whole_word(train_phrase))
test_2gram =  get_2gram(get_whole_word(test_phrase))

In [13]:
def sentence2idx(sentence,idx_list):
    try:
        return[idx_list[token] for token in sentence]
    except (KeyError,TypeError):
        # print(sentence)
        count = 0

### 加载数据

In [14]:
def load_data(train_phrase,train_label,batch_size):
    final_data = []
    batch_num = len(train_phrase)//batch_size
    for i in range(batch_num):
        batch_phrase = train_phrase[max(0,i*batch_size):min((i+1)*batch_size,len(train_phrase))]
        batch_label = train_label[max(0,i*batch_size):min((i+1)*batch_size,len(train_phrase))]
        ngram = []
        for sentence in batch_phrase:
            ngram.append(sentence2idx(sentence,char_to_idx))
        final_data.append((ngram,batch_label))
    return final_data

In [15]:
batch_size = 16
train_iter = load_data(train_2gram,train_label,batch_size)
test_iter = load_data(test_2gram,test_label,batch_size)

In [16]:
for x,y in train_iter:
    print(x)
    break

[[29645], [3091], [87148, 87149], [9056], [734, 735, 2746, 2747, 12707], [64703, 15890], [173, 3008, 27122], [5211, 4960, 5212, 2289, 5213, 5214, 3445, 3446, 5215, 5216, 5217, 5218, 5219, 5220, 5221, 5222, 5223], [21373, 21374, 7734, 21375, 21376, 21377, 21378, 21379, 1467, 1130, 9555, 21380, 9973, 21381, 21382], [61806, 17866, 61807, 61808], [1986, 1987, 13119, 13120, 6001, 452, 2227, 78649, 78650, 78651, 78652, 47388, 78653, 78654, 78655, 1909, 31708, 19042], [8068, 788, 2532], [64100, 35056, 64101, 56602], [97416, 70021, 97417, 2130, 57854, 59567, 1736, 37659, 97418], [83786], [42273, 94962, 62382, 94963, 4664, 41693, 4405, 18848, 27393, 25102, 94964, 35938, 94965, 94966, 49829, 19, 511, 2960, 77427, 17799, 94967, 94968, 46841, 7540, 25467, 94969, 94970, 94971, 94972, 14627]]


## 建模

In [17]:
import numpy as np 
from tqdm import tqdm 
import time

### Softmax实现

In [18]:
def softmax(x_input):
    x_input_exp = np.exp(x_input)
    partion = np.sum(x_input_exp,axis=1,keepdims=True)
    return x_input_exp/partion

### 小批量初始化

In [19]:
def feature(x):
    batch_size=len(x)
    feature_size=len(idx_to_char)
    inputs=np.zeros((batch_size,feature_size))
    for b,i in enumerate(x):
        for idx in i:
            inputs[b][idx]=1
    return inputs

### 反向传播

In [20]:
def backward(x,probability,y):
    probability[range(probability.shape[0]), y]-=1
    dw=x.T.dot(probability)/batch_size #feature_size*n_class
    db=np.sum(probability,axis=0)/batch_size #n_class
    return dw,db

### 评估函数

In [21]:
def evaluate(test_iter,W,b):
    right=0.0
    n=0.0
    for x,y in test_iter:
        n+=batch_size
        x=feature(x)
        probability=softmax(np.matmul(x,W)+b)
        right+=np.sum(np.argmax(probability,axis=1)==y)
    return right/n

### 训练函数

In [22]:
def train(train_data,test_data,lr,num_epoch,W,b,batch_size):
    for epoch in range(num_epoch):
        l_sum,start,n=0.0,time.time(),0
        train_iter=iter(train_data)
        test_iter=iter(test_data)
        for x,y in tqdm(train_iter):
            x=feature(x) #[batch_size,feature]
            probability=softmax(np.matmul(x,W)+b) #[batch_size,n_class]
            loss= np.sum(-np.log(probability[range(probability.shape[0]), y]))
            grad_w,grad_b=backward(x,probability,y)
            #print(x.shape,probability.shape,loss.shape,grad_w.shape,grad_b.shape)
            #print(grad_w,grad_b)
            W=W-lr*grad_w
            b=b-lr*grad_b
            l_sum+=loss
            n+=1
            #print(loss)
        print("epoch %d ,loss %.3f ,test_acc %.2f,time %.2f"%(epoch+1,l_sum/n,evaluate(test_iter,W,b),time.time()-start))
    return W,b

## 训练

In [28]:
batch_size=64
train_iter=load_data(train_2gram,train_label,batch_size)
test_iter=load_data(test_2gram,test_label,batch_size)
feature_size=len(idx_to_char)
n_class=5
W=np.random.normal(0,0.01,(feature_size,n_class))
b=np.zeros(n_class)
lr,num_epoch=2,10
print(train_iter[0])

([[29645], [3091], [87148, 87149], [9056], [734, 735, 2746, 2747, 12707], [64703, 15890], [173, 3008, 27122], [5211, 4960, 5212, 2289, 5213, 5214, 3445, 3446, 5215, 5216, 5217, 5218, 5219, 5220, 5221, 5222, 5223], [21373, 21374, 7734, 21375, 21376, 21377, 21378, 21379, 1467, 1130, 9555, 21380, 9973, 21381, 21382], [61806, 17866, 61807, 61808], [1986, 1987, 13119, 13120, 6001, 452, 2227, 78649, 78650, 78651, 78652, 47388, 78653, 78654, 78655, 1909, 31708, 19042], [8068, 788, 2532], [64100, 35056, 64101, 56602], [97416, 70021, 97417, 2130, 57854, 59567, 1736, 37659, 97418], [83786], [42273, 94962, 62382, 94963, 4664, 41693, 4405, 18848, 27393, 25102, 94964, 35938, 94965, 94966, 49829, 19, 511, 2960, 77427, 17799, 94967, 94968, 46841, 7540, 25467, 94969, 94970, 94971, 94972, 14627], [23477, 73595, 73596], [31469, 31470], [95364, 42490, 95365, 95366, 95367, 154, 643, 95368, 80485, 5534], [66223, 46938, 43801, 154, 35204], [96040, 96041], [526, 2440, 37093, 7154], [27896, 27897, 2358, 27898

In [29]:
train(train_iter,test_iter,lr,num_epoch,W,b,batch_size)

2023it [00:36, 56.15it/s]


epoch 1 ,loss 74.248 ,test_acc 0.58,time 39.93


2023it [00:39, 51.03it/s]


epoch 2 ,loss 64.036 ,test_acc 0.59,time 43.52


2023it [00:39, 50.70it/s]


...

2023it [00:39, 51.40it/s]


epoch 9 ,loss 46.281 ,test_acc 0.63,time 43.16


2023it [00:39, 50.93it/s]


epoch 10 ,loss 45.066 ,test_acc 0.63,time 43.62


(array([[-0.08647244, -0.51775476,  0.61073518, -0.07400414,  0.06984512],
        [ 0.39195092,  0.00178828, -0.11357876, -0.25443868, -0.02323411],
        [-0.00272594,  0.22928693, -0.1477594 , -0.06485332, -0.00716567],
        ...,
        [-0.0074    , -0.0362683 ,  0.09948904, -0.03943964, -0.01867478],
        [-0.01533404, -0.03966703,  0.09706932, -0.04298233,  0.00543835],
        [-0.02363159, -0.03340608,  0.09609087, -0.03272271, -0.01144012]]),
 array([-1.57507385,  0.11534904,  2.08727295,  0.52331056, -1.15085869]))